In [7]:
import numpy as np
import enum
# maybe I don't need hidden layers

In [128]:
class Utils:
    @classmethod
    def relu(cls, x):
        # x is expected to be a numpy vector
        x[x < 0] = 0
        return x
    
    @classmethod
    def sigmoid(cls, x):
        return 1/(1+np.exp(-x))
    
    @classmethod
    def add_bias_term(cls, vec):
        if len(vec.shape) < 2:
            vec = vec.reshape((1,-1))
        new_vec = np.ones((1,len(vec[0])+1))
        new_vec[0, :len(vec[0])] = vec[0]
        return new_vec.reshape((1,-1))
    
    @classmethod
    def genetic_crossover(cls, genome):
            rand = np.random.random()
            if 0 < rand < 0.45:
                return 100.0
            if 0.45 < rand < 0.9:
                return 99.0
            return np.random.randn()

In [149]:
class BirdNN:
    activations_supported = ("relu")
    input_dim = 3
    output_dim = 1
    
    def __init__(self, hidden_layers=None, inner_dims=None, activation=None, weights=None):
        self.hidden_layers = hidden_layers
        self.inner_dims = inner_dims
        self.activation = activation
        self.weights = weights
        
        if self.hidden_layers is None:
            self.hidden_layers = 1
        if self.inner_dims is None:
            self.inner_dims = [6]
        if self.activation is None:
            self.activation = 'relu'
        if self.weights is None:
            self.weights = self._create_random_network()
            
        self.check_params()
        self.activation_function = self._assign_activation_function()
        
    def check_params(self):
        if self.hidden_layers == 0:
            raise ValueError("hidden layers cannot be 0")
        if self.hidden_layers != len(self.inner_dims):
            raise ValueError("mismatch between hidden_layers and provided dimension sizes")
        if self.weights is not None and len(self.weights) != (self.hidden_layers+1):
            raise ValueError("weight matrixes provided do not match dimensions specified for Neural Network")
        if self.activation.lower() not in self.activations_supported:
            raise ValueError("activation function specified not supported")
    
    def _assign_activation_function(self):
        functions = {'relu':Utils.relu}
        return functions.get(self.activation)
        
    def _create_random_network(self):
        dims = [self.input_dim, *self.inner_dims , self.output_dim]
        dims = [(dims[i-1], dims[i]) for i in range(1, len(dims))]
        weights = []
        for matsize in dims:
            weights.append(self._create_random_weight_matrix(matsize[0], matsize[1]))
        return weights
            
    def forward(self, input_vec):
        current_vec = input_vec
        for weight_mat_ind in range(len(self.weights)):
            weight_mat = self.weights[weight_mat_ind]
            current_vec = Utils.add_bias_term(current_vec)
            # (1,n+1) * (n+1, m) = (1, m)
            current_vec = np.dot(current_vec, weight_mat)
            if weight_mat_ind < len(self.weights)-1:
                current_vec = self.activation_function(current_vec)
        return Utils.sigmoid(current_vec)
            
    def _create_random_weight_matrix(self, input_dim, output_dim):
        # the '+1' accounts for bias!
        W = np.random.normal(0,1, size=(input_dim+1, output_dim))
        # normalize just in case
        MAX = np.max(W)
        W = W/MAX
        return W

    
    def mate(self, otherNN):
        # assume that the inputs for the current input and the next input will be the same:
        child_weights = []
        for i in range(len(self.weights)):
            this_layer = self.weights[i]
            other_layer = otherNN.weights[i]
            dim = this_layer.shape
            child_layer, mask = np.zeros(dim), np.zeros(dim)
            crossover_func = np.vectorize(Utils.genetic_crossover)
            mask = crossover_func(mask)
            
            # DNA from parent A
            this_layer[mask<100.0] = 0
            child_layer += this_layer
            
            # DNA from parent B
            other_layer[mask!=99.0] = 0
            child_layer += other_layer
            
            # DNA from mutation
            mask[mask>=99.0] = 0
            child_layer += mask
            
            child_weights.append(child_layer)
        return BirdNN(hidden_layers=self.hidden_layers, inner_dims=self.inner_dims, activation="relu", weights=child_weights)
            
        

In [151]:
testNN = BirdNN(hidden_layers=2, inner_dims=[10,5])
otherNN = BirdNN(hidden_layers=2, inner_dims=[10,5])
newNN = testNN.mate(otherNN)
print(testNN.weights)
print(testNN)
test_vec = np.array([1,2,3])
testNN.forward(test_vec)

[(3, 10), (10, 5), (5, 1)]
[(3, 10), (10, 5), (5, 1)]
[[-8.66487816e-01  8.48079117e-02 -2.50514229e-01 -1.17696829e+00
  -6.13797160e-01  7.57051671e-01  8.83459925e-02  2.83209410e-01
  -9.28098254e-01 -5.23830267e-01]
 [ 3.34676407e-01 -8.19165436e-01 -1.20960832e+00  5.67322767e-01
  -3.59970280e-01 -7.12553649e-01  4.91696277e-01  4.94097520e-01
   1.43657334e-01 -6.42928662e-01]
 [ 8.22063657e-01 -6.38095536e-01 -1.33720752e+00 -2.00108361e-01
   2.79832910e-01 -6.69469130e-01 -3.22020314e-01  5.52148641e-01
   6.93392712e-01  9.20303033e-02]
 [-1.30964531e-03  3.69886468e-01  1.00000000e+00 -7.16232143e-01
   4.84150415e-01  9.38632957e-03 -1.45805531e+00  5.00367858e-01
   7.88598350e-01 -8.30368156e-01]]
[[ 0.63544314  0.02319004 -0.11957278  0.84582108  0.70722481 -0.21893225
   0.33303729  0.18258239  0.07174661 -0.68268968]
 [ 1.          0.5042925  -0.20224053  0.65976837 -0.44818943 -0.20978262
  -0.173611   -0.38237652  0.35092115 -0.17100093]
 [ 0.51081121  0.52309672 -

array([[0.64439175]])

In [68]:
Utils.relu(np.array([-1,-2,3,5]))

array([0, 0, 3, 5])

#### Todo:
- remove `hidden_layers` metric

In [111]:
def breed_func(x):
    rand = np.random.random()
    if 0 < rand < 0.45:
        return 100.0
    if 0.45 < rand < 0.9:
        return 99.0
    return np.random.randn()

f = np.vectorize(breed_func)
f(np.random.randn(10).astype('float'))

array([100.        , 100.        ,   0.64963185, 100.        ,
        99.        ,  -1.09500723,   0.31867489,  99.        ,
       100.        ,  99.        ])

In [103]:
np.random.randn()

-1.2568669539478057

In [121]:
a = np.identity(3)
a[a==1] = 3
a

array([[3., 0., 0.],
       [0., 3., 0.],
       [0., 0., 3.]])

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])